# A0149963M
## Assignment 4
## YSC2229 - Introductory Data Structures and Algorithms

Solution for Problem 1 - Runway Scheduling System

**1(a) - Store an approved/valid request in O(log n), where the stored information includes: aircraft ID (with format "AA123456") and take-off time**

This question has many parts. For now, I wil define the following:

In [38]:
Red = 'Red'
Black = 'Black'

In [39]:
class RBNode():
    def __init__(self, id_no = None, time = None):
        self.time = time
        self.id_no = id_no
        self.p = None
        self.left = None
        self.right = None
        self.col = Red 

In [40]:
class RedBlackTree():
    def __init__(self):
        self.NIL = RBNode()
        self.NIL.col = Black
        self.NIL.left = None
        self.NIL.right = None
        self.root = self.NIL
    
    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.left:
            x.p.left = y
        else:
            x.p.right = y
        y.left = x
        x.p = y
    
    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.right:
            x.p.right = y
        else: 
            x.p.left = y
        y.right = x
        x.p = y
    
        
    def fix_insert(self, z):
        while z.p.col == Red:
            if z.p == z.p.p.left:
                y = z.p.p.right
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.right:
                        z = z.p
                        self.left_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.right_rotate(z.p.p)
            else:
                y = z.p.p.left
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.left:
                        z = z.p
                        self.right_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.left_rotate(z.p.p)
            
        self.root.col = Black
      
        
    def insert(self, id_key, time_key):
        z = RBNode(id_key, time_key)
        
        y = self.NIL
        x = self.root
        
        while x != self.NIL:
            y = x
            if z.time < x.time:
                x = x.left
            else: 
                x = x.right
            
        z.p = y
        if y == self.NIL:
            self.root = z
        elif z.time < y.time:
            y.left = z
        else: 
            y.right = z
        
        z.left = self.NIL
        z.right = self.NIL
        z.col = Red
        
        self.fix_insert(z)
        
        
    def find_min(self):
        node = self.root
        while node.left != self.NIL:
            node = node.left
        print("Minimum time is:", node.time)
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def find_max(self):
        node = self.root
        while node.right != self.NIL:
            node = node.right
        print("Maximum time is:", node.time)
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def print_node(self, x):
        print("ID: ", x.id_no, ", Time: ", x.time)
        
    
    def inorder(self, node):
        if node != self.NIL:
            self.inorder(node.left)
            self.print_node(node)
            self.inorder(node.right)
    
    def search(self):
        pass

Testing functions

In [41]:
t = RedBlackTree()
t.insert("AR190237", 13)
t.insert("AS123984", 21)
t.insert("AE123973", 19)
t.insert("AR812731", 15)
t.insert("AS239183", 14)

t.find_min()
t.find_max()

t.inorder(t.root)


Minimum time is: 13
Corresponding ID number is: AR190237 

Maximum time is: 21
Corresponding ID number is: AS123984 

ID:  AR190237 , Time:  13
ID:  AS239183 , Time:  14
ID:  AR812731 , Time:  15
ID:  AE123973 , Time:  19
ID:  AS123984 , Time:  21


Solution for Problem 2 - Belief Propogation

Solution for Problem 3 - Huffman Code

page 430, 154

In [ ]:
class HuffmanNode:
    def __init__(self, freq, char = None, left = None, right = None):
        self.left = left
        self.right = right
        self.char = char
        self.freq = freq
        self.huff = ''

In [5]:
import sys

class MinPriorityQueue:
    def __init__(self, capacity):
        self.limit = capacity 
        self.size = 0
        self.Heap = [None] * (capacity + 1)
        self.Heap[0] = sys.maxsize * -1
        self.root = 1
    
    def swap(self, pos1, pos2):
        self.Heap[pos1], self.Heap[pos2] = self.Heap[pos2], self.Heap[pos1]
        
    def parent(self, pos):
        return pos // 2
    
    def left(self, pos):
        return pos * 2
    
    def right(self, pos):
        return pos * 2 + 1
    
    def min_heapify(self, pos):
        l = self.left(pos)
        r = self.right(pos)
        if l <= self.size and self.Heap[l] < self.Heap[pos]:
            low = l
        else:
            low = pos
        if r <= self.size and self.Heap[r] < self.Heap[low]:
            low = r
        if low != i:
            self.swap(i, low)
            self.min_heapify(low)
        
    def build_minheap(self):
        for i in range(self.size // 2, 0, -1):
            self.min_heapify(i)
            
    def heap_min(self):
        return self.Heap[1]
            
    def extract_min(self):
        if self.size < 1:
            raise ValueError("Heap Underflow")
        low = self.heap[1]
        self.Heap[1] = self.Heap[self.size]
        self.size -= 1
        self.min_heapify(1)
        return low
    
    def decrease_key(self,i, key):
        if key > self.Heap[i]:
            raise ValueError("New key Larger than Current key")
        self.Heap[i] = key
        while i > 1 and self.Heap[self.parent(i)] > self.Heap[i]:
            self.swap(self.parent(i), i)
            i = self.parent(i)
            
    def insert(self,key):
        self.size += 1
        self.Heap[self.size] = sys.maxsize
        self.decrease_key(self.size, key)
    
    def print_heap(self):
        for i in range(1, (self.size // 2) + 1):
            print("Node " + str(i) + " is "+ str(self.Heap[i]) +
                                     ". Left Child is " + str(self.Heap[2 * i]) + 
                                     ", Right Child is "+ str(self.Heap[2 * i + 1]))
   

In [6]:
minHeap = MinPriorityQueue(10)
minHeap.insert(15)
minHeap.insert(7)
minHeap.insert(9)
minHeap.insert(4)
minHeap.insert(8)
minHeap.insert(13)
minHeap.insert(70)


minHeap.print_heap()


Node 1 is 4. Left Child is 7, Right Child is 9
Node 2 is 7. Left Child is 15, Right Child is 8
Node 3 is 9. Left Child is 13, Right Child is 70


In [77]:
class HuffmanNode:
    def __init__(self, freq, char = None, left = None, right = None):
        self.left = left
        self.right = right
        self.char = char
        self.freq = freq
        self.huff = ''
    
    

In [5]:
def char_count(s):
    seen = {}
    for c in s:
        if c not in seen:
            seen[c] = 1
        else:
            seen[c] += 1
    return seen


def huffman_code(arr):
    """
    arr - array of sentences
    """
    for sentence in arr:
        
        table = char_count(sentence)
        q = MinPriorityQueue(len(table.keys()))
        
        
        
        
        


In [6]:
huffman_code(["88888888889999999999yyyyyyyyhhhhhhhjjjjj..../321zx"])

Solution for Problem 4 - Maximum Flow/ Minimum Cut Problem